In [ ]:
!ls ./

In [ ]:
flist = ['97家有喜事','唐伯虎點秋香v1','武狀元蘇乞兒','食神','功夫',
         '國產凌凌漆','鹿鼎記神龍教','唐伯虎秋香v2','少林足球','逃學威龍']

seg= [(4,2), (4,2), (5,2), (5,2),]

In [ ]:
def is_number(s):
    s=s.strip()
    try:
        int(s)
        return True
    except ValueError:
        return False

In [199]:
fw = open('resut.txt','w')
for fname  in flist:
    
    fread = open(fname)
    lines = fread.readlines()
    print fname,mod,ii
    
    start_index =0
    end_index = 0
    for index,l in enumerate(lines):
        if is_number(l): 
#             print index
            start_index = index
        elif l=='\n':
#             print index , '####'
            end_index = index
            
            l = ''
            
            if fname == '武狀元蘇乞兒':
                l = lines[start_index+2]
            else:
                for shift in range(end_index-start_index-2):
                    l+=lines[start_index+2+shift].strip()
            
            
        

            for r in ['》','《','「','」','-','∮','（','）','“','”','∼','"'] :
                l = l.replace(r,'')
            for r in [' ','…',',','，','、'] : # 逗點
                l = l.replace(r,'DOT')
            l = l.replace('DOTDOT','DOT')
            l = l.replace('DOTDOT','DOT')
            l = l.replace('DOTDOT','DOT')
            l = l.replace('DOT',' DOT ')
            
            l = l.replace('！','EXMARK')
            l = l.replace('!','EXMARK')

            
            l = l.replace('？','QMARK')
            l = l.replace('?','QMARK')
            resultl = ''
            for ss in [w for w in jieba.cut(l)] :
#                 print ss,
                resultl += ss+" "
            resultl = resultl.replace("  ",' ')
            resultl = resultl.replace("  ",' ')
            resultl = resultl.strip()
            fw.write(resultl.encode('utf-8') + "\r\n")
            fw.flush()
#             print
#             print
            
            
        
fw.close()

97家有喜事 5 2
唐伯虎點秋香v1 5 2
武狀元蘇乞兒 5 2
食神 5 2
功夫 5 2
國產凌凌漆 5 2
鹿鼎記神龍教 5 2
唐伯虎秋香v2 5 2
少林足球 5 2
逃學威龍 5 2


# 讀取檔案

In [204]:
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz ' # space is included in whitelist
EN_BLACKLIST = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\''

FILENAME = './resut.txt'

limit = {
        'maxq' : 20,
        'minq' : 0,
        'maxa' : 20,
        'mina' : 3
        }

UNK = 'unk'
VOCAB_SIZE = 30000

import random
import sys

import nltk
import itertools
from collections import defaultdict

import numpy as np

import pickle


def ddefault():
    return 1

'''
 read lines from file
     return [list of lines]

'''
def read_lines(filename):
    lines = open(filename).readlines()
    return [l.strip() for l in lines]


'''
 split sentences in one line
  into multiple lines
    return [list of lines]

'''
def split_line(line):
    return line.split('.')


'''
 remove anything that isn't in the vocabulary
    return str(pure ta/en)

'''
# def filter_line(line, whitelist):
#     return ''.join([ ch for ch in line if ch in whitelist ])


'''
 read list of words, create index to word,
  word to index dictionaries
    return tuple( vocab->(word, count), idx2w, w2idx )

'''
def index_(tokenized_sentences, vocab_size):
    # get frequency distribution
    freq_dist = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    # get vocabulary of 'vocab_size' most used words
    vocab = freq_dist.most_common(vocab_size)
    # index2word
    index2word = ['_'] + [UNK] + [ x[0] for x in vocab ]
    # word2index
    word2index = dict([(w,i) for i,w in enumerate(index2word)] )
    return index2word, word2index, freq_dist


'''
 filter too long and too short sequences
    return tuple( filtered_ta, filtered_en )

'''
def filter_data(sequences):
    filtered_q, filtered_a = [], []
    raw_data_len = len(sequences)

    for i in range(0, len(sequences)-1, 1):
        qlen, alen = len(sequences[i].split(' ')), len(sequences[i+1].split(' '))
        if qlen >= limit['minq'] and qlen <= limit['maxq']:
            if alen >= limit['mina'] and alen <= limit['maxa']:
                filtered_q.append(sequences[i])
                filtered_a.append(sequences[i+1])

    # print the fraction of the original data, filtered
    filt_data_len = len(filtered_q)
    filtered = int((raw_data_len - filt_data_len)*100/raw_data_len)
    print(str(filtered) + '% filtered from original data')

    return filtered_q, filtered_a





'''
 create the final dataset : 
  - convert list of items to arrays of indices
  - add zero padding
      return ( [array_en([indices]), array_ta([indices]) )
 
'''
def zero_pad(qtokenized, atokenized, w2idx):
    # num of rows
    data_len = len(qtokenized)

    # numpy arrays to store indices
    idx_q = np.zeros([data_len, limit['maxq']], dtype=np.int32) 
    idx_a = np.zeros([data_len, limit['maxa']], dtype=np.int32)

    for i in range(data_len):
        q_indices = pad_seq(qtokenized[i], w2idx, limit['maxq'])
        a_indices = pad_seq(atokenized[i], w2idx, limit['maxa'])

        #print(len(idx_q[i]), len(q_indices))
        #print(len(idx_a[i]), len(a_indices))
        idx_q[i] = np.array(q_indices)
        idx_a[i] = np.array(a_indices)

    return idx_q, idx_a


'''
 replace words with indices in a sequence
  replace with unknown if word not in lookup
    return [list of indices]

'''
def pad_seq(seq, lookup, maxlen):
    indices = []
    for word in seq:
        if word in lookup:
            indices.append(lookup[word])
        else:
            indices.append(lookup[UNK])
    return indices + [0]*(maxlen - len(seq))


def process_data():

    print('\n>> Read lines from file')
    lines = read_lines(filename=FILENAME)

    # change to lower case (just for en)
    lines = [ line.lower() for line in lines ]

    print('\n:: Sample from read(p) lines')
    for w in lines[:10]:
        print '##{}##'.format(w)

    # filter out too long or too short sequences
    print('\n>> 2nd layer of filtering')
    qlines, alines = filter_data(lines)
    print('\nq : {0} ; a : {1}'.format(qlines[60], alines[60]))
    print('\nq : {0} ; a : {1}'.format(qlines[61], alines[61]))


    # convert list of [lines of text] into list of [list of words ]
    print('\n>> Segment lines into words')
    qtokenized = [ wordlist.split(' ') for wordlist in qlines ]
    atokenized = [ wordlist.split(' ') for wordlist in alines ]
    print('\n:: Sample from segmented list of words')


    for w in qtokenized[60]:
        print w,
    print
    for w in atokenized[60]:
        print w,
    print

    # indexing -> idx2w, w2idx : en/ta
    print('\n >> Index words')
    idx2w, w2idx, freq_dist = index_( qtokenized + atokenized, vocab_size=VOCAB_SIZE)

    print('\n >> Zero Padding')
    idx_q, idx_a = zero_pad(qtokenized, atokenized, w2idx)

    print('\n >> Save numpy arrays to disk')
    # save them
    np.save('idx_q.npy', idx_q)
    np.save('idx_a.npy', idx_a)

    # let us now save the necessary dictionaries
    metadata = {
            'w2idx' : w2idx,
            'idx2w' : idx2w,
            'limit' : limit,
            'freq_dist' : freq_dist
                }

    # write to disk : data control dictionaries
    with open('metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)

def load_data(PATH=''):
    # read data control dictionaries
    with open(PATH + 'metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    # read numpy arrays
    idx_q = np.load(PATH + 'idx_q.npy')
    idx_a = np.load(PATH + 'idx_a.npy')
    return metadata, idx_q, idx_a


In [206]:
print('\n>> Read lines from file')
lines = read_lines(filename=FILENAME)

# change to lower case (just for en)
lines = [ line.lower() for line in lines ]

print('\n:: Sample from read(p) lines')
for w in lines[:10]:
    print '##{}##'.format(w)

# filter out too long or too short sequences
print('\n>> 2nd layer of filtering')
qlines, alines = filter_data(lines)
print('\nq : {0} ; a : {1}'.format(qlines[60], alines[60]))
print('\nq : {0} ; a : {1}'.format(qlines[61], alines[61]))


# convert list of [lines of text] into list of [list of words ]
print('\n>> Segment lines into words')
qtokenized = [ wordlist.split(' ') for wordlist in qlines ]
atokenized = [ wordlist.split(' ') for wordlist in alines ]
print('\n:: Sample from segmented list of words')


for w in qtokenized[60]:
    print w,
print
for w in atokenized[60]:
    print w,
print

# indexing -> idx2w, w2idx : en/ta
print('\n >> Index words')
idx2w, w2idx, freq_dist = index_( qtokenized + atokenized, vocab_size=VOCAB_SIZE)

print('\n >> Zero Padding')
idx_q, idx_a = zero_pad(qtokenized, atokenized, w2idx)

print('\n >> Save numpy arrays to disk')
# save them
np.save('idx_q.npy', idx_q)
np.save('idx_a.npy', idx_a)

# let us now save the necessary dictionaries
metadata = {
        'w2idx' : w2idx,
        'idx2w' : idx2w,
        'limit' : limit,
        'freq_dist' : freq_dist
            }

# write to disk : data control dictionaries
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)


>> Read lines from file

:: Sample from read(p) lines
##白粥 dot 油條 是 公公 的##
##乖乖 的 dot 多 喝點 dot 多吃點##
##火腿 煎 雙蛋加 報紙 dot 這是 老公 的##
##老外 玉米片 少點 糖加 百事 可樂 是 二叔 的##
##你 哪位 呀 qmark##
##tequila##
##無所謂 dot 起床##
##我 先去 洗個 臉##
##算了吧 dot 你 洗 不 洗 臉 都 是 這副 德性##
##你 什麼 意思 啊 exmark 你 昨晚 還說 我 漂亮 呢 exmark##

>> 2nd layer of filtering
13% filtered from original data

q : 拜 託 你 抽 時間 到 我 公司 去 dot ; a : 你別 胡說 八道 了 dot 我 前年 剛剛 才 去過

q : 你別 胡說 八道 了 dot 我 前年 剛剛 才 去過 ; a : 你還 知不知道 那 公司 在 哪 qmark

>> Segment lines into words

:: Sample from segmented list of words
拜 託 你 抽 時間 到 我 公司 去 dot
你別 胡說 八道 了 dot 我 前年 剛剛 才 去過

 >> Index words

 >> Zero Padding

 >> Save numpy arrays to disk
